In [28]:
%pip install polyline
%pip install geopy

Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 4.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [37]:
import requests
import json
import polyline
from geopy.distance import distance

In [70]:
#Valhalla URL
url = "http://192.168.1.138:8002/"
routeCosting = "auto"
unit = "miles"

def getRoute(start, end):
    data = {
        "locations":[
            {"lat": start[0], "lon": start[1]},
            {"lat": end[0], "lon": end[1]}
        ],
        "costing": routeCosting,
        "costing_options":{
            "auto":{
                "use_highways": 0,
                "use_tracks": 0.75 #default 0.5
            }
        },
        "directions_options": {
            "units": unit
        }
    }
    response = requests.post(url+'route', data=json.dumps(data))
    return response.text

def exportGEOJSON(response):
    dict = json.loads(response)
    shape = dict["trip"]["legs"][0]["shape"]
    polyline.decode(shape, 6, geojson=true)


def getWayID(wktShape):
    encoded = polyline.encode(wktShape,6)
    data = {
        "encoded_polyline": encoded,
        "costing": routeCosting,
        "directions_options": {
            "units": unit
        },
        "filters":{
            "attributes":[
            "edge.way_id"
            ],
            "action": "include"
        }
    }
    response = requests.post(url+'trace_attributes', data=json.dumps(data))
    dict = json.loads(response.text)
    id_list = []
    for w in dict["edges"]:
        id_list.append(str(w["way_id"]))
    #print("Found {0} ways".format(len(id_list)))
    id_list = list(dict.fromkeys(id_list))
    return id_list


def splitShape(shape, segmentKMs=195):
    shape = polyline.decode(shape,6)
    segments = []
    segmentStart = 0
    accumulated_distance = 0
    for i in range(1, len(shape)):
        segDistance = distance(shape[i-1],shape[i]).kilometers
        if accumulated_distance + segDistance < segmentKMs:
            accumulated_distance += segDistance
        else:
            segments.append(shape[segmentStart:i-1])
            segmentStart = i-1
            accumulated_distance = 0
    segments.append(shape[segmentStart:])
    return segments


In [71]:
locations = {
    #brooklyn
    "start": [40.68980,-73.97798],
    #salt lake city
    "end": [40.76698,-111.83501]
}
result = getRoute(locations["start"],locations["end"])
print("Found route")
resultDict = json.loads(result)
shape = resultDict["trip"]["legs"][0]["shape"]
segments = splitShape(shape)
print("Split into {0} segments".format(len(segments)))
wayIDs = []
for s in segments:
    wayIDs.extend(getWayID(s))
print("found {0} ways".format(len(wayIDs)))
print(",".join(wayIDs))

Found route
Split into 26 segments
found 5342 ways
903142088,493933674,421118561,420512619,420512627,110155730,954645297,511768217,349154287,456391963,1049645588,954644445,420571176,974011826,420571178,524345883,5677896,357279521,46189305,46189291,492441172,46189288,531546246,19725512,464963297,420355081,675967321,997125811,165777195,496989303,165777194,505772014,439842754,165777255,420355080,439842751,165777635,420355079,439842761,985415393,439842757,165777951,276670360,945523294,630828592,584971395,438389785,945473887,561084838,496992369,5671774,1040863520,814612602,489019754,221157083,496986905,388240124,587061869,554083353,587038101,1106484060,729972919,729972918,1087744935,458166897,710571388,465000799,587038082,1250866958,505778913,497165757,497165754,388696206,1087048475,5670139,497258109,497258108,1211049195,420899372,420899371,1205900284,196116988,195743152,420899373,186202420,1211612938,439801903,171574337,1024425029,91383966,630828582,671202178,420276995,1161025024,835176560